In [6]:
# Install necessary libraries
!pip install transformers torchaudio

In [7]:

# Import required modules
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
from google.colab import files


In [16]:
from getpass import getpass
token = getpass('Enter Hugging Face Token')
%env HF_TOKEN=$token


Enter Hugging Face Token··········
env: HF_TOKEN=dataset = load_my_dataset()  # This function should return an iterable of data suitable for training  model.train()  for batch in dataset:     input_values = processor(batch["input_audio"], return_tensors="pt").input_values     labels = torch.tensor(batch["labels"])  # Make sure your labels are correctly formatted as tensors     outputs = model(input_values, labels=labels)     loss = outputs.loss     loss.backward()     optimizer.step()     optimizer.zero_grad()


In [17]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


model.train()




Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [18]:

def predict(audio_path):
    # Load the audio file
    speech_array, sampling_rate = torchaudio.load(audio_path)

    # Resample if necessary
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        speech_array = resampler(speech_array)

    # Prepare the input features for the model
    inputs = processor(speech_array[0], sampling_rate=16000, return_tensors="pt", padding=True, return_attention_mask=True)

    # Prepare arguments for the model
    input_values = inputs.input_values
    attention_mask = inputs.get('attention_mask', None)

    # Use the model to predict the transcript
    with torch.no_grad():
        model_args = {'input_values': input_values}
        if attention_mask is not None:
            model_args['attention_mask'] = attention_mask
        logits = model(**model_args).logits

    # Decode the model output
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription


In [19]:


# Upload file
uploaded = files.upload()

# Assuming the uploaded file is named correctly after checking
if uploaded:
    file_name = list(uploaded.keys())[0]  # This will use the actual name of the uploaded file
    print("Uploaded file name:", file_name)
    # Use the predict function
    transcription = predict(file_name)
    print("Transcription:", transcription)
else:
    print("No file uploaded.")


Saving harvard.wav to harvard (2).wav
Uploaded file name: harvard (2).wav
Transcription: THE STAALE SMEL OF E OL BAR LIGERS I TAKES HEAT TO BRING OUT THE  ODOR A COLD DIP RESTORES HEATH AND ZEST A SLT PICKL TATSFINE HAM TACOES AL PASTOE AR MY FAOURITE A ZESTFU FOOD IS THE OT COSS BUN
